# 自定义块
实现 `MLP`

In [3]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F

In [4]:
class MLP(nn.Module):
    # 用模型声明参数层。这里声明两个全连接的层
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256) # hidden layer
        self.out = nn.Linear(256,10) # output layer
        
    # 定义模型的前向传播，及如何根据输入X返回模型所需输出
    def forward(self,X):
        # 这里使用ReLU的函数版本
        return self.out(F.relu(self.hidden(X)))

前向传播 `forward` 函数以 `X` 作为输入，计算带有激活函数的隐藏表示，并输出未经规范的值

In [5]:
X = torch.rand(2,20)

In [6]:
# 调用如下
net = MLP()
net(X)

tensor([[-0.2521,  0.1947, -0.0405,  0.0290,  0.3063,  0.1302,  0.2003, -0.1790,
         -0.1646, -0.0712],
        [-0.3734,  0.1326, -0.0074, -0.0895,  0.2730,  0.1599,  0.1811, -0.1447,
         -0.0854, -0.0537]], grad_fn=<AddmmBackward0>)

# 顺序块
构造自己的 `Sequential`
- 将块逐个追加到列表的函数
- 前向传播函数，将输入按追加块的顺序传递给组成的 `block chain` 

In [7]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx,module in enumerate(args):
            # module是Module子类的一个实例
            # 变量 _modules中，module的类型是 OrderedDict
            self._modules[str(idx)] = module
            
    def forward(self,X):
        # OrderedDict保证了按照成员的添加顺序来遍历
        for block in self._modules.values():
            X = block(X)
        return X

`__init__` 将每个模块添加到有序字典中。
之所以每个 `Module` 都有一个 `_modules` 属性，而不是自己定义一个列表，是因为在模块参数初始化的过程中，系统知道去 `_modules` 字典中查找需要初始化参数的子块

In [9]:
# 调用如下
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0994,  0.0099, -0.1042, -0.0203, -0.0349,  0.2844,  0.0382, -0.1087,
         -0.0545,  0.2849],
        [ 0.1341, -0.0249, -0.0727, -0.1132, -0.1912,  0.1848, -0.0537, -0.0392,
          0.1031,  0.4374]], grad_fn=<AddmmBackward0>)

# 合并常数参数
常数参数：既不是上一层的输出，也不是可更新的项，但是需要合并到当前层中


In [10]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 常量参数，不计算梯度，不会更新，训练期间保持不变
        self.rand_weight = torch.randn(20,20,requires_grad=False)
        self.linear = nn.Linear(20,20)
    
    def forward(self,X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        # 复用全连接层，相当于两个全连接层共享参数
        X = self.linear(X)
        # Python控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

`self.rand_weight` 是一个常量参数，不会计算梯度。此外代码还实现了前向传播中执行Python控制流

In [11]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1134, grad_fn=<SumBackward0>)

# 混搭组合块
- 嵌套块

In [17]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                               nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))

In [18]:
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(0.1100, grad_fn=<SumBackward0>)

# 自定义层
- 不带参数的层
- 带参数的层

In [19]:
import torch
import torch.nn.functional as F
from torch import nn


In [21]:
# 不带参数的层
class CenteredLayer(nn.Module):
    def __init_(self):
        super().__init__()
        
    def forward(self,X):
        return X - X.mean()

In [23]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [24]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [26]:
Y = net(torch.rand(8))
Y.mean()
# 得到的是接近于0的数，因为浮点数精度问题

tensor(6.0536e-09, grad_fn=<MeanBackward0>)

# 带参数的层
可以使用内置函数来创建参数，这些函数提供一些基本功能，比如管理访问、初始化、共享、保存和加载参数，这样做的好处是：不需要为每一个自定义层编写自定义的序列化程序

In [32]:
class MyLinear(nn.Module):
    def __init__(self,in_units,units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units,units))
        self.bias = nn.Parameter(torch.randn(units))
        
    def forward(self,X):
        linear = torch.matmul(X,self.weight.data) + self.bias.data
        return F.relu(linear)

In [33]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[-0.6689, -0.2611, -0.7655],
        [ 2.0279, -0.3501,  0.8894],
        [ 0.3324,  0.8740, -0.9742],
        [ 1.2870,  0.2596, -0.6359],
        [-1.3969, -1.2201,  0.4012]], requires_grad=True)

In [37]:
net = nn.Sequential(MyLinear(5,3),MyLinear(3,1))
net(torch.rand(1,5))

tensor([[1.8840]])